In [ ]:
#install the Pyterrier framework
!pip install python-terrier
# install the nltk modules
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup

In [ ]:
import pyterrier as pt

if not pt.started():
  # In this lab, we need to specify that we start PyTerrier with PRF enabled
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



In [ ]:
# Import Libraries

import re
import pandas as pd
import nltk
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

from nltk.stem import *
from nltk.stem.porter import *
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [ ]:
corpus_directory = '/content/corpus.jsonl'

In [ ]:
import pandas as pd
import os

# Function to ensure directory exists
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to convert JSONL to CSV
def jsonl_to_csv(jsonl_file_path, csv_file_path):
    ensure_dir(csv_file_path)
    df = pd.read_json(jsonl_file_path, lines=True)
    df.to_csv(csv_file_path, index=False)

# Function to convert TSV to CSV
def tsv_to_csv(tsv_file_path, csv_file_path):
    ensure_dir(csv_file_path)
    df = pd.read_csv(tsv_file_path, sep='\t')
    df.to_csv(csv_file_path, index=False)

# Paths for JSONL files
corpus_jsonl_path = '/content/corpus.jsonl'
queries_jsonl_path = '/content/queries.jsonl'

# Paths for TSV files
test_tsv_path = '/content/test.tsv'
train_tsv_path = '/content/train.tsv'

# Output CSV paths
corpus_csv_path = '/mnt/data/corpus.csv'
queries_csv_path = '/mnt/data/queries.csv'
test_csv_path = '/mnt/data/test.csv'
train_csv_path = '/mnt/data/train.csv'

# Convert files
jsonl_to_csv(corpus_jsonl_path, corpus_csv_path)
jsonl_to_csv(queries_jsonl_path, queries_csv_path)
tsv_to_csv(test_tsv_path, test_csv_path)
tsv_to_csv(train_tsv_path, train_csv_path)

In [ ]:
df=pd.read_csv("/mnt/data/corpus.csv")
df

,_id,title,text,metadata
0,MED-10,Statin Use and Breast Cancer Survival: A Natio...,"Recent studies have suggested that statins, an...",{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/25...
1,MED-14,Statin use after diagnosis of breast cancer an...,BACKGROUND: Preclinical studies have shown tha...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/25...
2,MED-118,Alkylphenols in human milk and their relations...,The aims of this study were to determine the c...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/20...
3,MED-301,Methylmercury: A Potential Environmental Risk ...,Epilepsy or seizure disorder is one of the mos...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/22...
4,MED-306,Sensitivity of Continuous Performance Test (CP...,Hit Reaction Time latencies (HRT) in the Conti...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/20...
...,...,...,...,...
3628,MED-917,Effect of freezing and storage on the phenolic...,Scottish-grown red raspberries are a rich sour...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/12...
3629,MED-941,Topical vitamin A treatment of recalcitrant co...,BACKGROUND: Common warts (verruca vulgaris) ar...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed?te...
3630,MED-942,Esophageal injury by apple cider vinegar table...,Apple cider vinegar products are advertised in...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/15...
3631,MED-952,Cannabis and the lung.,The use of cannabis is embedded within many so...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/21...


In [ ]:
#Import the necessary modules:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re # used to clean the data
#to display the full text on the notebook without truncation
pd.set_option('display.max_colwidth', 150)

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.stem import *
from nltk.stem.porter import *

In [ ]:
# Initialize Porter stemmer
stemmer = PorterStemmer()

In [ ]:
def Steem_text(text):

    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    # print (tokens)
    return ' '.join(stemmed_tokens)

In [ ]:
# Function to remove stopwords
def remove_stopwords(text):

    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words] #Lower is used to normalize al the words make them in lower case
    print('Tokens are:',tokens,'\n')
    return ' '.join(filtered_tokens)

In [ ]:
import pandas as pd

In [ ]:
#the docno will be our tweetID
df["docno"]=df["_id"].astype(str)
df[["docno"]]

,docno
0,MED-10
1,MED-14
2,MED-118
3,MED-301
4,MED-306
...,...
3628,MED-917
3629,MED-941
3630,MED-942
3631,MED-952


In [ ]:
def clean(text):
   text = re.sub(r"http\S+", " ", text) # remove urls
   text = re.sub(r"RT ", " ", text) # remove rt
   text = re.sub(r"@[\w]*", " ", text) # remove handles
   text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text) # remove special characters
   text = re.sub(r'\t', ' ', text) # remove tabs
   text = re.sub(r'\n', ' ', text) # remove line jump
   text = re.sub(r"\s+", " ", text) # remove extra white space
   text = text.strip()
   return text

In [ ]:
df['processed_title'] = df['title'].apply(clean)
df

,_id,title,text,metadata,docno,processed_title
0,MED-10,Statin Use and Breast Cancer Survival: A Nationwide Cohort Study from Finland,"Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast...",{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/25329299'},MED-10,Statin Use and Breast Cancer Survival A Nationwide Cohort Study from Finland
1,MED-14,Statin use after diagnosis of breast cancer and survival: a population-based cohort study.,"BACKGROUND: Preclinical studies have shown that statins, particularly simvastatin, can prevent growth in breast cancer cell lines and animal model...",{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/25304447'},MED-14,Statin use after diagnosis of breast cancer and survival a population-based cohort study
2,MED-118,Alkylphenols in human milk and their relations to dietary habits in central Taiwan.,The aims of this study were to determine the concentrations of 4-nonylphenol (NP) and 4-octylphenol (OP) in 59 human milk samples and to examine r...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/20435081%20'},MED-118,Alkylphenols in human milk and their relations to dietary habits in central Taiwan
3,MED-301,Methylmercury: A Potential Environmental Risk Factor Contributing to Epileptogenesis,Epilepsy or seizure disorder is one of the most common neurological diseases in humans. Although genetic mutations in ion channels and receptors a...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/22206970'},MED-301,Methylmercury A Potential Environmental Risk Factor Contributing to Epileptogenesis
4,MED-306,Sensitivity of Continuous Performance Test (CPT) at Age 14 Years to Developmental Methylmercury Exposure,Hit Reaction Time latencies (HRT) in the Continuous Performance Test (CPT) measure the speed of visual information processing. The latencies may i...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/20699117'},MED-306,Sensitivity of Continuous Performance Test (CPT) at Age 14 Years to Developmental Methylmercury Exposure
...,...,...,...,...,...,...
3628,MED-917,"Effect of freezing and storage on the phenolics, ellagitannins, flavonoids, and antioxidant capacity of red raspberries.","Scottish-grown red raspberries are a rich source of vitamin C and phenolics, most notably, the anthocyanins cyanidin-3-sophoroside, cyanidin-3-(2(...",{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/12188629'},MED-917,Effect of freezing and storage on the phenolics ellagitannins flavonoids and antioxidant capacity of red raspberries
3629,MED-941,Topical vitamin A treatment of recalcitrant common warts.,BACKGROUND: Common warts (verruca vulgaris) are benign epithelial proliferations associated with human papillomavirus (HPV) infection. Salicylic a...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed?term=cider%5BAll%20Fields%5D%20AND%20%28%22acetic%20acid%22%5BMeSH%20Terms%5D%20OR%20%28%22acetic%22%5...,MED-941,Topical vitamin A treatment of recalcitrant common warts
3630,MED-942,Esophageal injury by apple cider vinegar tablets and subsequent evaluation of products.,Apple cider vinegar products are advertised in the popular press and over the Internet for treatment of a variety of conditions. After an adverse ...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/15983536'},MED-942,Esophageal injury by apple cider vinegar tablets and subsequent evaluation of products
3631,MED-952,Cannabis and the lung.,"The use of cannabis is embedded within many societies, mostly used by the young and widely perceived to be safe. Increasing concern regarding the ...",{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/21132143'},MED-952,Cannabis and the lung


In [ ]:
df['processed_text'] = df['text'].apply(clean)
df

,_id,title,text,metadata,docno,processed_title,processed_text
0,MED-10,Statin Use and Breast Cancer Survival: A Nationwide Cohort Study from Finland,"Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast...",{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/25329299'},MED-10,Statin Use and Breast Cancer Survival A Nationwide Cohort Study from Finland,Recent studies have suggested that statins an established drug group in the prevention of cardiovascular mortality could delay or prevent breast c...
1,MED-14,Statin use after diagnosis of breast cancer and survival: a population-based cohort study.,"BACKGROUND: Preclinical studies have shown that statins, particularly simvastatin, can prevent growth in breast cancer cell lines and animal model...",{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/25304447'},MED-14,Statin use after diagnosis of breast cancer and survival a population-based cohort study,BACKGROUND Preclinical studies have shown that statins particularly simvastatin can prevent growth in breast cancer cell lines and animal models W...
2,MED-118,Alkylphenols in human milk and their relations to dietary habits in central Taiwan.,The aims of this study were to determine the concentrations of 4-nonylphenol (NP) and 4-octylphenol (OP) in 59 human milk samples and to examine r...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/20435081%20'},MED-118,Alkylphenols in human milk and their relations to dietary habits in central Taiwan,The aims of this study were to determine the concentrations of 4-nonylphenol (NP) and 4-octylphenol (OP) in 59 human milk samples and to examine r...
3,MED-301,Methylmercury: A Potential Environmental Risk Factor Contributing to Epileptogenesis,Epilepsy or seizure disorder is one of the most common neurological diseases in humans. Although genetic mutations in ion channels and receptors a...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/22206970'},MED-301,Methylmercury A Potential Environmental Risk Factor Contributing to Epileptogenesis,Epilepsy or seizure disorder is one of the most common neurological diseases in humans Although genetic mutations in ion channels and receptors an...
4,MED-306,Sensitivity of Continuous Performance Test (CPT) at Age 14 Years to Developmental Methylmercury Exposure,Hit Reaction Time latencies (HRT) in the Continuous Performance Test (CPT) measure the speed of visual information processing. The latencies may i...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/20699117'},MED-306,Sensitivity of Continuous Performance Test (CPT) at Age 14 Years to Developmental Methylmercury Exposure,Hit Reaction Time latencies (HRT) in the Continuous Performance Test (CPT) measure the speed of visual information processing The latencies may in...
...,...,...,...,...,...,...,...
3628,MED-917,"Effect of freezing and storage on the phenolics, ellagitannins, flavonoids, and antioxidant capacity of red raspberries.","Scottish-grown red raspberries are a rich source of vitamin C and phenolics, most notably, the anthocyanins cyanidin-3-sophoroside, cyanidin-3-(2(...",{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/12188629'},MED-917,Effect of freezing and storage on the phenolics ellagitannins flavonoids and antioxidant capacity of red raspberries,Scottish-grown red raspberries are a rich source of vitamin C and phenolics most notably the anthocyanins cyanidin-3-sophoroside cyanidin-3-(2(G)-...
3629,MED-941,Topical vitamin A treatment of recalcitrant common warts.,BACKGROUND: Common warts (verruca vulgaris) are benign epithelial proliferations associated with human papillomavirus (HPV) infection. Salicylic a...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed?term=cider%5BAll%20Fields%5D%20AND%20%28%22acetic%20acid%22%5BMeSH%20Terms%5D%20OR%20%28%22acetic%22%5...,MED-941,Topical vitamin A treatment of recalcitrant common warts,BACKGROUND Common warts (verruca vulgaris) are benign epithelial proliferations associated with human p

In [ ]:
df['processed_text'] = df['processed_text'].apply(remove_stopwords)
df['processed_title'] = df['processed_title'].apply(remove_stopwords)

Streaming output truncated to the last 5000 lines.
Tokens are: ['Prognostic', 'and', 'therapeutic', 'implications', 'of', 'mTORC1', 'and', 'Rictor', 'expression', 'in', 'human', 'breast', 'cancer'] 

Tokens are: ['Bean', 'and', 'rice', 'meals', 'reduce', 'postprandial', 'glycemic', 'response', 'in', 'adults', 'with', 'type', '2', 'diabetes', 'a', 'cross-over', 'study'] 

Tokens are: ['High', 'intake', 'of', 'whole', 'grains', 'and', 'beans', 'pattern', 'is', 'inversely', 'associated', 'with', 'insulin', 'resistance', 'in', 'healthy', 'Korean', 'adult', 'population'] 

Tokens are: ['Antidiabetic', 'drugs', 'used', 'in', 'Europe', 'prior', 'to', 'the', 'discovery', 'of', 'insulin'] 

Tokens are: ['Beans', 'and', 'diabetes', 'Phaseolus', 'vulgaris', 'preparations', 'as', 'antihyperglycemic', 'agents'] 

Tokens are: ['Effect', 'of', 'non-oil-seed', 'pulses', 'on', 'glycaemic', 'control', 'a', 'systematic', 'review', 'and', 'meta-analysis', 'of', 'randomised', 'controlled', 'experimental', 

In [ ]:
df['processed_text'] = df['processed_text'].apply(Steem_text)
df['processed_title'] = df['processed_title'].apply(Steem_text)
df

,_id,title,text,metadata,docno,processed_title,processed_text
0,MED-10,Statin Use and Breast Cancer Survival: A Nationwide Cohort Study from Finland,"Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast...",{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/25329299'},MED-10,statin use breast cancer surviv nationwid cohort studi finland,recent studi suggest statin establish drug group prevent cardiovascular mortal could delay prevent breast cancer recurr effect disease-specif mort...
1,MED-14,Statin use after diagnosis of breast cancer and survival: a population-based cohort study.,"BACKGROUND: Preclinical studies have shown that statins, particularly simvastatin, can prevent growth in breast cancer cell lines and animal model...",{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/25304447'},MED-14,statin use diagnosi breast cancer surviv population-bas cohort studi,background preclin studi shown statin particularli simvastatin prevent growth breast cancer cell line anim model investig whether statin use breas...
2,MED-118,Alkylphenols in human milk and their relations to dietary habits in central Taiwan.,The aims of this study were to determine the concentrations of 4-nonylphenol (NP) and 4-octylphenol (OP) in 59 human milk samples and to examine r...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/20435081%20'},MED-118,alkylphenol human milk relat dietari habit central taiwan,aim studi determin concentr 4-nonylphenol ( np ) 4-octylphenol ( op ) 59 human milk sampl examin relat factor includ mother ' demograph dietari ha...
3,MED-301,Methylmercury: A Potential Environmental Risk Factor Contributing to Epileptogenesis,Epilepsy or seizure disorder is one of the most common neurological diseases in humans. Although genetic mutations in ion channels and receptors a...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/22206970'},MED-301,methylmercuri potenti environment risk factor contribut epileptogenesi,epilepsi seizur disord one common neurolog diseas human although genet mutat ion channel receptor risk factor brain injuri link epileptogenesi und...
4,MED-306,Sensitivity of Continuous Performance Test (CPT) at Age 14 Years to Developmental Methylmercury Exposure,Hit Reaction Time latencies (HRT) in the Continuous Performance Test (CPT) measure the speed of visual information processing. The latencies may i...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/20699117'},MED-306,sensit continu perform test ( cpt ) age 14 year development methylmercuri exposur,hit reaction time latenc ( hrt ) continu perform test ( cpt ) measur speed visual inform process latenc may involv differ neuropsycholog function ...
...,...,...,...,...,...,...,...
3628,MED-917,"Effect of freezing and storage on the phenolics, ellagitannins, flavonoids, and antioxidant capacity of red raspberries.","Scottish-grown red raspberries are a rich source of vitamin C and phenolics, most notably, the anthocyanins cyanidin-3-sophoroside, cyanidin-3-(2(...",{'url': 'http://www.ncbi.nlm.nih.gov/pubmed/12188629'},MED-917,effect freez storag phenol ellagitannin flavonoid antioxid capac red raspberri,scottish-grown red raspberri rich sourc vitamin c phenol notabl anthocyanin cyanidin-3-sophorosid cyanidin-3- ( 2 ( g ) -glucosylrutinosid ) cyani...
3629,MED-941,Topical vitamin A treatment of recalcitrant common warts.,BACKGROUND: Common warts (verruca vulgaris) are benign epithelial proliferations associated with human papillomavirus (HPV) infection. Salicylic a...,{'url': 'http://www.ncbi.nlm.nih.gov/pubmed?term=cider%5BAll%20Fields%5D%20AND%20%28%22acetic%20acid%22%5BMeSH%20Terms%5D%20OR%20%28%22acetic%22%5...,MED-941,topic vitamin treatment recalcitr common wart,background common wart ( verruca vulgari ) benign epitheli prolifer associ human papillomaviru ( hpv ) infect salicyl acid cryotherapi frequent tr...
3630,MED-942,Esophageal injury by apple cider vinegar tablets and subsequent evaluation of produc

In [ ]:
indexer = pt.DFIndexer("./myFirstIndex", overwrite=True)

# index the text, record the docnos as metadata
index_ref = indexer.index(df["processed_text"], df["processed_title"], df["docno"])

In [ ]:
print(index_ref.toString())
#we will first load the index
index = pt.IndexFactory.of(index_ref)
#we will call getCollectionStatistics() to check the stats
print(index.getCollectionStatistics().toString())

./myFirstIndex/data.properties
Number of documents: 3633
Number of terms: 17957
Number of postings: 313276
Number of fields: 0
Number of tokens: 485126
Field names: []
Positions:   false



In [ ]:
for kv in index.getLexicon():
  print("%s -> %s " % (kv.getKey(), kv.getValue().toString()))

Streaming output truncated to the last 5000 lines.
phenelzin -> term4590 Nt=1 TF=1 maxTF=1 @{0 265133 5} 
phenethyl -> term10617 Nt=3 TF=3 maxTF=1 @{0 265135 5} 
phenobarbit -> term11883 Nt=2 TF=2 maxTF=1 @{0 265141 7} 
phenol -> term944 Nt=91 TF=187 maxTF=9 @{0 265147 3} 
phenolcarbon -> term14354 Nt=1 TF=1 maxTF=1 @{0 265264 5} 
phenomena -> term6560 Nt=6 TF=6 maxTF=1 @{0 265267 5} 
phenomenolog -> term6275 Nt=4 TF=5 maxTF=2 @{0 265281 3} 
phenomenon -> term4724 Nt=21 TF=24 maxTF=3 @{0 265289 6} 
phenotyp -> term2351 Nt=30 TF=38 maxTF=4 @{0 265329 1} 
phentolamin -> term13083 Nt=1 TF=1 maxTF=1 @{0 265374 5} 
phenyl -> term10033 Nt=5 TF=6 maxTF=2 @{0 265377 3} 
phenylalanin -> term7869 Nt=4 TF=10 maxTF=6 @{0 265388 2} 
phenylalanyl -> term7872 Nt=1 TF=1 maxTF=1 @{0 265396 6} 
phenylazo -> term11377 Nt=1 TF=1 maxTF=1 @{0 265399 2} 
phenylazophenol -> term11380 Nt=1 TF=1 maxTF=1 @{0 265402 0} 
phenylendiamin -> term14985 Nt=1 TF=1 maxTF=1 @{0 265404 6} 
phenylhydrazin -> term16967 Nt=1 

Query Processing

In [ ]:
#we need to process the query also as we did for documents
def preprocess(sentence):
  sentence = remove_stopwords(sentence)
  sentence = clean(sentence)
  sentence = Steem_text(sentence)

  return sentence

In [ ]:
query="study"
query = preprocess(query)
query

Tokens are: ['study'] 



'studi'

In [ ]:
splited_query = query.split()
len(splited_query)

1

In [ ]:
# Identify the documents that have the query

# Function(1) : Split the documents into tokens

def split_doc_to_tokens(text):

  docs = {}

  i = 0

  for doc in text:
    docs[i] = doc.split()

    i = i + 1

  splited_docs_val = docs.values()

  return splited_docs_val


# Function(2) : return the the docs that have the same token with query

def num_of_doc_of_same_query(spli):

 lis = []

 for i in range(len(splited_query)):

   doc_num = 0

   for j in spli:

     doc_num = doc_num + 1

     for k in range(len(j)):

       if splited_query[i] == j[k]:
         lis.append(doc_num)

 return lis


# Function(3) : the all function

def docs_IDs(text):
  split_docs = split_doc_to_tokens(text)

  nums = num_of_doc_of_same_query(split_docs)

  return nums


def Retrieve_docs(dfs , text):
  split_doc = split_doc_to_tokens(text)

  numo = num_of_doc_of_same_query(split_doc)

  for d in numo:
    # Printing value of column 'A' at index 2
    print(f"Document Number {d}: \n {dfs['processed_text'].iloc[d]}")



splited_docs_values = docs_IDs(df['processed_text'])

print(splited_docs_values)

[1, 1, 2, 3, 6, 6, 6, 7, 8, 8, 8, 8, 8, 9, 10, 10, 13, 14, 14, 14, 16, 16, 18, 18, 21, 21, 22, 22, 22, 22, 23, 26, 26, 27, 30, 30, 34, 34, 35, 36, 38, 38, 41, 41, 45, 48, 48, 51, 51, 52, 52, 52, 52, 52, 54, 55, 55, 56, 57, 58, 58, 59, 62, 62, 62, 63, 63, 64, 65, 65, 66, 68, 68, 68, 68, 69, 70, 70, 72, 73, 73, 73, 73, 74, 77, 77, 78, 79, 80, 81, 82, 82, 82, 82, 83, 83, 83, 85, 86, 86, 86, 87, 87, 88, 90, 91, 92, 92, 92, 92, 92, 94, 94, 96, 98, 98, 98, 98, 98, 100, 101, 105, 106, 107, 107, 107, 107, 108, 110, 110, 110, 110, 110, 110, 110, 112, 113, 114, 114, 114, 115, 115, 115, 117, 118, 118, 119, 119, 120, 120, 121, 123, 123, 123, 126, 126, 126, 126, 126, 126, 128, 129, 129, 130, 130, 131, 132, 132, 132, 132, 132, 133, 134, 138, 138, 138, 138, 138, 139, 142, 142, 143, 143, 145, 145, 145, 146, 148, 150, 151, 152, 153, 154, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 156, 156, 158, 158, 159, 159, 159, 159, 159, 159, 159, 159, 159, 15

In [ ]:
# Rank the retrievd documents based ranking algorithm (TF-IDF)

tfidf_retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})

In [ ]:
  results = tfidf_retr.search(query)
  results

,qid,docid,docno,rank,score,query
0,1,3602,MED-5362,0,1.489940,studi
1,1,740,MED-1656,1,1.473612,studi
2,1,2503,MED-4097,2,1.470759,studi
3,1,1150,MED-2158,3,1.466164,studi
4,1,3564,MED-5323,4,1.457775,studi
...,...,...,...,...,...,...
995,1,2654,MED-4299,995,0.982331,studi
996,1,2856,MED-4543,996,0.982331,studi
997,1,1825,MED-3175,997,0.981323,studi
998,1,441,MED-1307,998,0.980360,studi


Query Expansion

In [ ]:
import pandas as pd
import pyterrier as pt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
import os
pd.set_option('display.max_colwidth', 150)

In [ ]:
# Need to install additional terrier package for PRF. It will take around 1 min
!git clone https://github.com/terrierteam/terrier-prf/
!apt-get install maven   #used for Java projects to manage project dependencies and build processes
%cd /content/terrier-prf/
!mvn install
!pwd
%cd ..

Cloning into 'terrier-prf'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 196 (delta 52), reused 173 (delta 36), pack-reused 0
Receiving objects: 100% (196/196), 28.00 KiB | 4.67 MiB/s, done.
Resolving deltas: 100% (52/52), done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libaopalliance-java libapache-pom-java libatinject-jsr330-api-java libcdi-api-java
  libcommons-cli-java libcommons-io-java libcommons-lang3-java libcommons-parent-java
  libgeronimo-annotation-1.3-spec-java libgeronimo-interceptor-3.0-spec-java libguava-java
  libguice-java libhawtjni-runtime-java libjansi-java libjansi-native-java libjsr305-java
  libmaven-parent-java libmaven-resolver-java libmaven-shared-utils-java libmaven3-core-java
  libplexus-cipher-java libplexus-classworlds-java libplexus-compone

In [ ]:
# Define our retrieval model
bm25 = pt.BatchRetrieve(index, wmodel="BM25",num_results=10)

result = bm25.search(query)
result

,qid,docid,docno,rank,score,query
0,1,1084,MED-2082,0,-0.295690,studi
1,1,3537,MED-5293,1,-0.409438,studi
2,1,132,MED-907,2,-0.424735,studi
3,1,2557,MED-4159,3,-0.479263,studi
4,1,3068,MED-4798,4,-0.495400,studi
5,1,3239,MED-4983,5,-0.504391,studi
6,1,2185,MED-3680,6,-0.510569,studi
7,1,3319,MED-5066,7,-0.522297,studi
8,1,2065,MED-3538,8,-0.525590,studi
9,1,460,MED-1328,9,-0.533436,studi


In [ ]:
df[['text']][df['docno'].isin(results['docno'].loc[0:4].tolist())]

,text
740,"Background Low back pain (LBP) is common in children and adolescents, and it is becoming a public health concern. In recent years there has been a..."
1150,Background Epidemiologic studies have reported inconsistent results regarding coffee consumption and the risk of liver cancer. We performed a meta...
2503,The identification of modifiable lifestyle factors that could reduce the risk of breast cancer is a research priority. Despite the enormous chemo ...
3564,This study reviewed the literature on the relations between exposure to chemicals with endocrine-disrupting abilities and obesity in humans. The s...
3602,"BACKGROUND: Studies of single nutrients on depression have produced inconsistent results, and they have failed to consider the complex interaction..."


In [ ]:
# "rewrite" function from PyTerrier will be used to expand queries specifying RM3 as the model
# fb_docs ==> no. expansion documents
# fb_terms ==> no. expansion terms
rm3_expander = pt.rewrite.RM3(index,fb_terms=10, fb_docs=100)

#output of the BM25 will be fed into the RM3 expander for query expansion.
rm3_qe = bm25 >> rm3_expander
expanded_query = rm3_qe.search(query).iloc[0]["query"]

expanded_query

'applypipeline:off sleep^0.059422966 de^0.041461289 hr^0.039546002 studi^0.600000024 2010^0.036808472 mortal^0.039610092 estrogen^0.031455129 death^0.044529110 le^0.039388224 acut^0.031150229 stroke^0.036628466'

In [ ]:
# Just print the expanded query with term scores
for s in expanded_query.split()[1:]:
  print(s)

print("\n" + query)

sleep^0.059422966
de^0.041461289
hr^0.039546002
studi^0.600000024
2010^0.036808472
mortal^0.039610092
estrogen^0.031455129
death^0.044529110
le^0.039388224
acut^0.031150229
stroke^0.036628466

studi


In [ ]:
# After that you can search using the expanded query
expanded_query_formatted = ' '.join(expanded_query.split()[1:])

results_wqe = bm25.search(expanded_query_formatted)

print("   Before Expansion    After Expansion")
print(pd.concat([results[['docid','score']][0:5].add_suffix('_1'),
            results_wqe[['docid','score']][0:5].add_suffix('_2')], axis=1).fillna(''))

#Let's check the tweets text for the top 5 retrieved tweets
df[['text']][df['docno'].isin(results_wqe['docno'].loc[0:5].tolist())]

   Before Expansion    After Expansion
   docid_1   score_1  docid_2   score_2
0     3602  1.489940     1402  2.214509
1      740  1.473612       43  2.155372
2     2503  1.470759     3068  1.920987
3     1150  1.466164     2961  1.914959
4     3564  1.457775     2558  1.743840


,text
43,Diverticular disease of the colon is among the most prevalent conditions in western society and is among the leading reasons for outpatient visits...
1084,"BACKGROUND: Reliable and timely information on the leading causes of death in populations, and how these are changing, is a crucial input into hea..."
1402,BACKGROUND AND METHODS: We estimated the prevalence of self-reported asthma in adult Indians and examined several risk factors influencing disease...
2558,"CONTEXT: Despite decades of accumulated observational evidence, the balance of risks and benefits for hormone use in healthy postmenopausal women ..."
2961,A widespread misconception has been developing among the Canadian public and among physicians. It is increasingly believed that consumption of die...
3068,OBJECTIVE To review the evidence for the efficacy of products used for environmental or hand cleaning on the rates of Clostridium difficile–associ...


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

#load the ELMo model
elmo = hub.load("https://tfhub.dev/google/elmo/3")

#define the sentences
queryyy1 = "Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality"
queryyy1 = preprocess(queryyy1)
queryyy2 = "Preclinical studies have shown that statins, particularly simvastatin, can prevent growth in breast cancer cell lines and animal model"
queryyy2 = preprocess(queryyy2)
sentences = [queryyy1, queryyy2]

#generate ELMo embeddings for the sentences
embeddings = elmo.signatures["default"](tf.constant(sentences))["elmo"]


studies_queryyy1_embedding = embeddings.numpy()[0][1]
studies_queryyy2_embedding = embeddings.numpy()[1][1]

#print the embeddings vectors
print("Embedding vector for 'studies' (queryyy1):", studies_queryyy1_embedding)
print("Embedding vector for 'studies' (queryyy2):", studies_queryyy2_embedding)

Tokens are: ['Recent', 'studies', 'have', 'suggested', 'that', 'statins', ',', 'an', 'established', 'drug', 'group', 'in', 'the', 'prevention', 'of', 'cardiovascular', 'mortality'] 

Tokens are: ['Preclinical', 'studies', 'have', 'shown', 'that', 'statins', ',', 'particularly', 'simvastatin', ',', 'can', 'prevent', 'growth', 'in', 'breast', 'cancer', 'cell', 'lines', 'and', 'animal', 'model'] 

Embedding vector for 'studies' (queryyy1): [-0.01276469 -0.50988436  0.54222023 ...  0.0457438   0.6397473
 -0.17128187]
Embedding vector for 'studies' (queryyy2): [ 0.07403457 -0.0079816   0.09402289 ... -0.17207573  0.2035615
  0.00318901]


In [ ]:
embeddings.numpy().shape

(2, 14, 1024)

UI

In [ ]:
!pip install flask_ngrok

In [ ]:
df2 = df.head(50)

df2 = df2.to_dict()

df2

{'_id': {0: 'MED-10',
  1: 'MED-14',
  2: 'MED-118',
  3: 'MED-301',
  4: 'MED-306',
  5: 'MED-329',
  6: 'MED-330',
  7: 'MED-332',
  8: 'MED-334',
  9: 'MED-335',
  10: 'MED-398',
  11: 'MED-557',
  12: 'MED-666',
  13: 'MED-691',
  14: 'MED-692',
  15: 'MED-702',
  16: 'MED-706',
  17: 'MED-707',
  18: 'MED-708',
  19: 'MED-709',
  20: 'MED-711',
  21: 'MED-712',
  22: 'MED-713',
  23: 'MED-714',
  24: 'MED-716',
  25: 'MED-717',
  26: 'MED-718',
  27: 'MED-719',
  28: 'MED-720',
  29: 'MED-721',
  30: 'MED-722',
  31: 'MED-723',
  32: 'MED-724',
  33: 'MED-726',
  34: 'MED-727',
  35: 'MED-728',
  36: 'MED-729',
  37: 'MED-730',
  38: 'MED-731',
  39: 'MED-732',
  40: 'MED-733',
  41: 'MED-734',
  42: 'MED-735',
  43: 'MED-736',
  44: 'MED-743',
  45: 'MED-744',
  46: 'MED-745',
  47: 'MED-746',
  48: 'MED-748',
  49: 'MED-749'},
 'title': {0: 'Statin Use and Breast Cancer Survival: A Nationwide Cohort Study from Finland',
  1: 'Statin use after diagnosis of breast cancer and survi

In [ ]:
def func(df2 , que):
 i = 0

 quer = preprocess(que)

 docs_id = []

 for key, value in df2.items():
   if key == 'processed_text':
         val = value.values()
         for doc in val:
           terms = doc.split()
           for term in terms:
             if term == quer and i not in docs_id:
               docs_id.append(f'''Document number {i} -----> \n{df["text"][i]}''')
           i = i + 1
 return docs_id

In [ ]:
query2 = "study"

x = func(df2 , query2)
x

Tokens are: ['study'] 



['Document number 0 -----> \nRecent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast cancer recurrence but the effect on disease-specific mortality remains unclear. We evaluated risk of breast cancer death among statin users in a population-based cohort of breast cancer patients. The study cohort included all newly diagnosed breast cancer patients in Finland during 1995–2003 (31,236 cases), identified from the Finnish Cancer Registry. Information on statin use before and after the diagnosis was obtained from a national prescription database. We used the Cox proportional hazards regression method to estimate mortality among statin users with statin use as time-dependent variable. A total of 4,151 participants had used statins. During the median follow-up of 3.25 years after the diagnosis (range 0.08–9.0 years) 6,011 participants died, of which 3,619 (60.2%) was due to breast cancer. After adjustme

In [ ]:
from google.colab.output import eval_js
print (eval_js("google.colab.kernel.proxyPort(5000)"))

https://oygmeu3i06l-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from flask import Flask, request
from flask_ngrok import run_with_ngrok

# Assuming you've already defined the sui function and imported necessary modules

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return """
    <style>
        body {
            background-color: white;
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
        }

        .header {
            background-color: black;
            color: white;
            padding: 20px 0;
        }

        .container {
            text-align: center;
        }

        h1 {
            text-align: center;
            margin: 0;
            padding: 10px 0;
        }

        #searchInput {
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 20px; /* Increased border-radius for a rounded appearance */
            margin-bottom: 10px;
            width: 300px; /* Adjust the width as needed */
            box-sizing: border-box; /* Include padding and border in the element's total width */
            transition: border-color 0.3s; /* Smooth transition for border color change */
        }

        #searchInput:focus {
            border-color: #007bff; /* Change border color on focus */
        }

        button {
            padding: 10px 20px;
            background-color: #007bff;
            color: white;
            border: none;
            border-radius: 20px; /* Increased border-radius for a rounded appearance */
            cursor: pointer;
            transition: background-color 0.3s; /* Smooth transition for background color change */
        }

        button:hover {
            background-color: #0056b3; /* Change background color on hover */
        }
    </style>

    <div class="header">
        <h1>Welcome to Kareem's Search Engine</h1>
    </div>
    <div class="container">
        <input type="text" id="searchInput" placeholder="Enter your query...">
        <button onclick="search()">Search</button>
    </div>
    <div id="searchResult"></div>

    <script>
        function search() {
            var searchTerm = document.getElementById("searchInput").value;
            fetch('/search', {
                method: 'POST',
                body: JSON.stringify({ query: searchTerm }),
                headers:{
                    'Content-Type': 'application/json'
                }
            })
            .then(response => response.json())
            .then(data => {
                console.log("Received data:", data); // Debug: Check if data is received
                var resultDiv = document.getElementById("searchResult");
                resultDiv.innerHTML = "<h2>Relevant Documents IDs:</h2>";
                if (data.results.length === 0) {
                    resultDiv.innerHTML += "<p>No documents found</p>";
                } else {
                    data.results.forEach(doc => {
                        console.log("Displaying document:", doc); // Debug: Check if document is displayed
                        resultDiv.innerHTML += "<p>" + doc + "</p>";
                    });
                }
            })
            .catch(error => {
                console.error('Error occurred during fetch:', error); // Debug: Log fetch errors
            });
        }
    </script>
    """

@app.route("/search", methods=['POST'])
def search():
    query = request.json['query']
    print("Received query:", query)  # Debug: Check if Flask receives the query
    results = func(df2, query)
    print("Search results:", results)  # Debug: Check if sui function returns results
    return {'results': results}

app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll

Evaluation

In [ ]:
import pandas as pd
import pyterrier as pt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
import os
pd.set_option('display.max_colwidth', 150)

In [ ]:
vaswani_dataset = pt.datasets.get_dataset("vaswani")

dff = vaswani_dataset.get_topics()

dff['docno'] = dff.index

# Rename column 'A' to 'X'
dff = dff.rename(columns={'query': 'Text'})

qrels = vaswani_dataset.get_qrels()

qrels['docno']=qrels['docno'].astype(str)

dff

query-text.trec:   0%|          | 0.00/3.05k [00:00<?, ?iB/s]

qrels:   0%|          | 0.00/6.63k [00:00<?, ?iB/s]

,qid,Text,docno
0,1,measurement of dielectric constant of liquids by the use of microwave techniques,0
1,2,mathematical analysis and design details of waveguide fed microwave radiations,1
2,3,use of digital computers in the design of band pass filters having given phase and attenuation characteristics,2
3,4,systems of data coding for information transfer,3
4,5,use of programs in engineering testing of computers,4
...,...,...,...
88,89,tunnel diode construction and its electrical characteristics explained,88
89,90,electronic density of states at the surface of a semiconductor compared with that at depth,89
90,91,resistivity of metallic thin films related to surface roughness,90
91,92,the phenomenon of radiation caused by charged particles moving in varying electric and magnetic fields,91


In [ ]:
indexref2 = vaswani_dataset.get_index()
index2 = pt.IndexFactory.of(indexref2)

print(index2.getCollectionStatistics().toString())

data.direct.bf:   0%|          | 0.00/388k [00:00<?, ?iB/s]

data.document.fsarrayfile:   0%|          | 0.00/234k [00:00<?, ?iB/s]

data.inverted.bf:   0%|          | 0.00/362k [00:00<?, ?iB/s]

data.lexicon.fsomapfile:   0%|          | 0.00/682k [00:00<?, ?iB/s]

data.lexicon.fsomaphash:   0%|          | 0.00/777 [00:00<?, ?iB/s]

...

data.meta.idx:   0%|          | 0.00/89.3k [00:00<?, ?iB/s]

data.meta.zdata:   0%|          | 0.00/224k [00:00<?, ?iB/s]

data.properties:   0%|          | 0.00/4.29k [00:00<?, ?iB/s]

md5sums:   0%|          | 0.00/619 [00:00<?, ?iB/s]

Number of documents: 11429
Number of terms: 7756
Number of postings: 224573
Number of fields: 1
Number of tokens: 271581
Field names: [text]
Positions:   false



In [ ]:
# qrels = pd.read_csv("/mnt/data/queries.csv")

In [ ]:
retr = pt.BatchRetrieve(index2, controls = {"wmodel": "TF_IDF"})

res = retr.search("mathematical")
res

,qid,docid,docno,rank,score,query
0,1,4746,4747,0,5.168347,mathematical
1,1,7399,7400,1,5.036916,mathematical
2,1,5629,5630,2,4.912003,mathematical
3,1,7997,7998,3,4.912003,mathematical
4,1,4546,4547,4,4.679886,mathematical
...,...,...,...,...,...,...
147,1,3484,3485,147,1.828498,mathematical
148,1,7283,7284,148,1.747822,mathematical
149,1,6714,6715,149,1.702745,mathematical
150,1,8622,8623,150,1.606095,mathematical


In [ ]:
eval = pt.Evaluate(res,qrels)
eval

{'map': 4.7960250544348844e-06, 'ndcg': 0.00022891881462746983}